In [6]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

YOUTUBE_VIDEO = "https://www.youtube.com/watch?v=BblV6AQsd2s&ab_channel=RiseAgainstVEVO"

### Setting up the model
Define the LLM model to use

In [7]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")

model.invoke("Are we in a simulation?")

AIMessage(content='There is currently no scientific evidence to definitively prove or disprove the idea that we are living in a simulation. The concept of a simulated reality is a philosophical and theoretical idea that has been explored by various thinkers and scientists, but it remains a topic of speculation and debate. Ultimately, the question of whether or not we are in a simulation may never be definitively answered.', response_metadata={'finish_reason': 'stop', 'logprobs': None})

### Using the magic of Chains :)

In [8]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

# Chain example
chain = model | parser

chain.invoke("Are we in a simulation?")

'There is no definitive answer to this question as it is still a topic of debate among scientists, philosophers, and theorists. Some believe that it is possible that we are living in a simulation created by a higher intelligence or advanced civilization, while others argue that there is no concrete evidence to support this theory. Ultimately, it is up to individual interpretation and belief.'

### Using prompt templates

In [9]:
from langchain_core.prompts import ChatPromptTemplate

template = """
  Answer the question based on the context below. If you can't
  answer the question, just say "I don't know".

  Context: {context}

  Question: {question}
  """

prompt = ChatPromptTemplate.from_template(template)
print(prompt.format(context="The sky is blue.", question="What color is the sky?"))

print("Did it work?")

chain = prompt | model | parser
chain.invoke({
  "context": "I have an appointment tomorrow at 3pm.", 
  "question": "Can I go for lunch tomorrow at 3:00PM?"
})

Human: 
  Answer the question based on the context below. If you can't
  answer the question, just say "I don't know".

  Context: The sky is blue.

  Question: What color is the sky?
  
Did it work?


"No, it's not advisable to go for lunch at 3:00PM since you have an appointment at that time."

### Combining Chains

In [10]:
from operator import itemgetter

translation_template = """
  Translate the 

  answer: {answer}

  to

  language: {language}
  """

translation_prompt = ChatPromptTemplate.from_template(translation_template)

translation_chain = (
  {"answer": chain, "language": itemgetter("language") } | translation_prompt | model | parser
)

translation_chain.invoke({
  "context": "I have an appointment tomorrow at 3pm.", 
  "question": "Can I go for lunch tomorrow at 3:00PM?",
  "language": "es"
})


'Respuesta: No, tienes una cita a las 3pm mañana.'

### Transcribing the YouTube Video

In [11]:
import tempfile
import whisper
from pytube import YouTube

if not os.path.exists("transcription.txt"):
  youtube = YouTube(YOUTUBE_VIDEO)
  audio = youtube.streams.filter(only_audio=True).first()

  whisper_model = whisper.load_model("base")

  with tempfile.TemporaryDirectory() as tmpdir:
    file = audio.download(output_path=tmpdir)
    transcroption = whisper_model.transcribe(file, fp16=False)["text"].strip()

    with open("transcription.txt", "w") as f:
      f.write(transcroption)

TypeError: argument of type 'NoneType' is not iterable